In [ ]:
FLIGHT_PER_DAY = 6

AVERAGE_CUSTOMER_PER_FLIGHT = 120
AVERAGE_SEAT_PER_ACCOUNT = 4
AVERAGE_ACCOUNT_PER_FLIGHT = AVERAGE_CUSTOMER_PER_FLIGHT / AVERAGE_SEAT_PER_ACCOUNT

DAYS_PER_YEAR = 365

YEARS = 3

PERCENTAGE_OF_DUPLICATES_ACCOUNT = 0.5

ACCOUNTS_NEEDED = FLIGHT_PER_DAY * DAYS_PER_YEAR * YEARS * AVERAGE_ACCOUNT_PER_FLIGHT * (1 - PERCENTAGE_OF_DUPLICATES_ACCOUNT)
ACCOUNTS_NEEDED

In [ ]:
from typing import NamedTuple
from datetime import datetime

class Account(NamedTuple):
    username: str
    hashed_password: str
    joined_date: datetime
    email_address: str
    phone_number: str
    gender: str
    

In [ ]:
import requests 
from typing import Iterable
# import bcrypt
from datetime import timedelta
import numpy as np
import random 
import time
 
def query_random_person(n: int = 1) -> Iterable[dict]:
    # ratelimit 20000 per minute
    k = 0
    for i in range(n // 5000 + 1):
        if k % 20000 == 0:
            time.sleep(120)
        
        ret = requests.get(f"https://randomuser.me/api/?results={min(n, 5000)}&format=json&noinfo") 
        ret.raise_for_status()
        yield from ret.json()["results"]
        k += 1
        
    
def get_random_time_offset(years: int = 3) -> timedelta:
    random_seconds = np.random.randint(0, years * 365 * 24 * 60 * 60)
    return timedelta(seconds=random_seconds)


def get_random_email_domain() -> str:
    choices = ["gmail", "hotmail", "outlook", "yahoo"]
    weights = [0.85, 0.02, 0.1, 0.03]
    return np.random.choice(choices, p=weights)


def generate_random_hash() -> str:
    BASE_64 = "./ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"
    # $2b$12$aIWCDww3E2lrCLY/rGWkrOxcgzKJ2R6cXFX/0jOpJ0Rdw.zZQNUPO
    return f"$2b$12${''.join(random.choices(BASE_64, k=53))}"


def generate_random_account(n: int) -> Iterable[Account]:
    now = datetime.now()
    for person in query_random_person(n):
        email_address = person["email"].replace("example", get_random_email_domain())
        if np.random.randint(0, 2) == 1:
            email_address = email_address[email_address.find(".") + 1:]
        yield Account(
            username=f"{person["name"]["first"]} {person["name"]["last"]}",
            # hashed_password=bcrypt.hashpw(person["login"]["password"].encode("utf-8"), bcrypt.gensalt()).decode("utf-8"),
            hashed_password=generate_random_hash(),
            joined_date=now - get_random_time_offset(),
            email_address=email_address,
            phone_number=person["phone"],
            gender=person["gender"][0].upper()
        )

trans = str.maketrans({"'": "''", "&": r"\&"})
make_sql_nice = lambda s: s.translate(trans)

with open("stuff.sql", "a") as f:
    for account in generate_random_account(int(ACCOUNTS_NEEDED) - 90000):
        sql_stmt = f"    INTO ACCOUNT (USERNAME, HASHED_PASSWORD, JOINED_DATE, EMAIL_ADDRESS, PHONE_NUMBER, GENDER) " \
              f"VALUES (" \
              f"'{make_sql_nice(account.username)}', " \
              f"'{make_sql_nice(account.hashed_password)}', " \
              f"TO_TIMESTAMP('{account.joined_date:%Y-%m-%d %H:%M:%S}', 'YYYY-MM-DD HH24:MI:SS'), " \
              f"'{make_sql_nice(account.email_address)}', " + \
              (f"'{account.phone_number}'" if np.random.randint(0, 5) != 1 else 'NULL') + ", " + \
              (f"'{account.gender}'" if np.random.randint(0, 10) != 1 else 'NULL') + \
              ")"
        print(sql_stmt)
        f.write(sql_stmt + '\n')
    